In [1]:
import requests
from bs4 import BeautifulSoup


In [2]:
from io import StringIO

In [305]:
import pandas as pd
import time

In [18]:
import sys,os
sys.path.append(os.getcwd())


In [63]:
police_url = "https://haryanapolice.gov.in/ViewFIR/FIRStatusSearch?From=LFhlihlx/W49VSlBvdGc4w=="

In [64]:
pool = requests.get(police_url)

In [33]:
test_url = "https://haryanapolice.gov.in/ViewFIR/FIRStatusSearch?From=LFhlihlx%2fW49VSlBvdGc4w%3d%3d"

In [275]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [378]:
service = Service("./chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--log-level=3")
options.add_argument("--headless")
driver = webdriver.Chrome(service=service, options=options)

In [379]:
def fetch_fir_reports(test_url, driver):
    
    driver.get(test_url)

    Select(
        driver.find_element(By.NAME, "ctl00$ContentPlaceHolder1$ddFIRYear")
    ).select_by_value("2019")

    Select(
        driver.find_element(By.NAME, "ctl00$ContentPlaceHolder1$ddlDistrict")
    ).select_by_value("13225")

    Select(
        driver.find_element(By.NAME, "ctl00$ContentPlaceHolder1$ddlPoliceStation")
    ).select_by_value("13225008")

    driver.find_element(
        By.XPATH,
        "//input[@name='ctl00$ContentPlaceHolder1$btnStatusSearch' and @value='Search']",
    ).click()

    tag_present = EC.presence_of_element_located((By.ID, "tblDisplayRecords"))
    wait = WebDriverWait(driver, 10)
    wait.until(tag_present)

    table_links = driver.find_elements(
        By.XPATH, '//table[@id="tblDisplayRecords"]/tbody/tr'
    )

    n = len(table_links)
    fir_texts = []
    first_page_data = []

    for index in range(n):
        print(index)
        
        fir = table_links[index]

        fir.find_elements(
            By.ID, f"ContentPlaceHolder1_rptDisplayRecords_lnkFIRViewDetail_{index}"
        )[0].click()

        driver.switch_to.window(driver.window_handles[1])

        tag_present = EC.presence_of_element_located((By.NAME, "RptView$ctl06$ctl00$Next$ctl00$ctl00"))
        wait = WebDriverWait(driver, 10)
        wait.until(tag_present)

        time.sleep(2)

        first_page_content = driver.find_element(
            By.XPATH, "//td[contains(@id, 'oReportCell')]"
        ).text
        first_page_data.append(first_page_content)
        
        time.sleep(5)
        driver.find_element(
            By.XPATH, "//input[@name='RptView$ctl06$ctl00$Next$ctl00$ctl00']"
        ).click()

        tag_present = EC.presence_of_element_located((By.XPATH, "//span[contains(@class, '556')]"))
        wait = WebDriverWait(driver, 10)
        wait.until(tag_present)
        time.sleep(2)
        
        full_fir_text = driver.find_element(
            By.XPATH, "//span[contains(@class, '556')]"
        ).text
        print(full_fir_text)

        fir_texts.append(full_fir_text)

        driver.switch_to.window(driver.window_handles[0])
        # refresh table
        table_links = driver.find_elements(
            By.XPATH, '//table[@id="tblDisplayRecords"]/tbody/tr'
        )

    return fir_texts, first_page_data


In [380]:
fir_texts, first_page_data = fetch_fir_reports(test_url, driver)

0
सेवा में, श्रीमान् SHO साहब थाना भुपानी जय हिन्द। आज मन ASI मय साथी C अरविन्द 3175 के बराये गस्त वा पडताल क्राईम महावतपुर से नचौली रोड पर मौजुद था कि मुखबर खास ने मुलाकी होकर बतलाया कि नचौली गांव के पुल के पास 2 व्यक्ति तास के पत्ते से जुआ खेल रहे है। जो मुखबर की बात को सच्ची मानकर मन ASI ने साथी C को समझाकर मुखबर को हमराह लेकर रवाना नचौली पुल का हुआ जहां पहुंच कर मुखबर ने दुर से ईशारा किया कि वे पुल के पास जुआ खेल रहे है मुखबर को वही फारिग करके मन ASI ने साथी C की ईमदाद से पुल के पास छिप कर देखा तो 2 व्यक्ति पुल के पास जुआ खेल रहे थे एक व्यक्ति ने कहा मेरी 50 रुपये की चाल दुसरे ने कहा मेरी 100 रुपये की चाल आई जो मन ASI ने जुआ खेलते हुए व्यक्तियो को काबू किया नाम पता पुछा तो पहले ने अपना नाम बिनेश S/O मामचन्द जाति गुज्जर गांव नचौली थाना भुपानी फरीदाबाद वा दुसरे ने अपना नाम सागर S/O संजय जाति बाल्मिकी गांव महावतपुर थाना भुपानी फरीदाबाद बतलाया जो बिनेश के बाये गोडे के नीचे 200/रुपये वा सागर के दांए गोडे के नीचे 150/रुपये मिले पीड में से लेकर गिनती करने पर 450 रुपये व 52 पत्ते ताश वा अख

In [385]:
[x for x in fir_texts if "25-" in x]

['सेवा मे, श्रीमान SHO साहब थाना भुपानी ‘जय हिंद’ आज मन HC मय सि0 सोनु 2785 व मय गाडी सरकारी HR-51 BM – 8073 टाटा सुमो जिसका चालक CT. जगदीप 1205 है के बराये गश्त व पडताल क्राईम खेडी पुल-नचौली रोड भुपानी मोड पर मौजुद हुँ कि एक नौजवान लडका गांव भुपानी की तरफ से पैदल आता हुआ दिखाई दिया जो सामने खडी पुलिस पार्टी को देखकर एक दम वापिस मुडकर तेज कदमो से चलने लगा। मन HC ने भागकर साथी CT की मदद से शक की बिनाह पर शख्स मजकुरा को काबु करके नाम पता पुछा तो जिसने अपना नाम रोहित S/O स्व0 जितेन्द्र भारती R/O रामनगर कालोनी भुपानी मोड थाना भुपानी जिला फरीदाबाद बा उम्र 22 वर्ष बतलाया। तलाशी लेने पर जिसकी पहनी हुई पैंट जींस की दाहिनी डब से एक देशी पिस्तौल 32 बोर बरामद हुआ। चैक करने पर जो खाली मिला। शख्स मजकुरा अपने पास देशी पिस्तौल रखने बारे मांगने पर कोई लाईंसेंस पेश नही कर सका। देशी पिस्तौल का अलग के खाका तैयार करके लाईसेंस पेश नही कर सका। देशी पिस्तौल का अलग से खाका तैयार करके पैमाइश करने पर स्लाईड बैरल ल0 14 C.M, बट ल0 8 CM व कुल लम्बाई 18 C.M हुई। बरामदा शुदा देशी पिस्तौल का अलग से पलंदा तैयार करके प

In [394]:
fir_texts[2]

'सेवा मे, श्रीमान SHO साहब थाना भुपानी ‘जय हिंद’ आज मन HC मय सि0 सोनु 2785 व मय गाडी सरकारी HR-51 BM – 8073 टाटा सुमो जिसका चालक CT. जगदीप 1205 है के बराये गश्त व पडताल क्राईम खेडी पुल-नचौली रोड भुपानी मोड पर मौजुद हुँ कि एक नौजवान लडका गांव भुपानी की तरफ से पैदल आता हुआ दिखाई दिया जो सामने खडी पुलिस पार्टी को देखकर एक दम वापिस मुडकर तेज कदमो से चलने लगा। मन HC ने भागकर साथी CT की मदद से शक की बिनाह पर शख्स मजकुरा को काबु करके नाम पता पुछा तो जिसने अपना नाम रोहित S/O स्व0 जितेन्द्र भारती R/O रामनगर कालोनी भुपानी मोड थाना भुपानी जिला फरीदाबाद बा उम्र 22 वर्ष बतलाया। तलाशी लेने पर जिसकी पहनी हुई पैंट जींस की दाहिनी डब से एक देशी पिस्तौल 32 बोर बरामद हुआ। चैक करने पर जो खाली मिला। शख्स मजकुरा अपने पास देशी पिस्तौल रखने बारे मांगने पर कोई लाईंसेंस पेश नही कर सका। देशी पिस्तौल का अलग के खाका तैयार करके लाईसेंस पेश नही कर सका। देशी पिस्तौल का अलग से खाका तैयार करके पैमाइश करने पर स्लाईड बैरल ल0 14 C.M, बट ल0 8 CM व कुल लम्बाई 18 C.M हुई। बरामदा शुदा देशी पिस्तौल का अलग से पलंदा तैयार करके पि

In [395]:
first_page_data[2]

"HARYANA POLICE CITIZEN SERVICES (हरियाणा पुलिस नागरिक सेवा)\n    FIRST INFORMATION REPORT\n         (Under Section 154 Cr.P.C.)\n        प्रथम सूचना रिपोर्ट\n (धारा 154 दंड प्रक्रिया सहिंता के तहत)\n1.\nDistrict (ज़िला): FARIDABAD\nP.S. (थाना): BHUPANI\nYear (वर्ष): 2019\nFIR No. (प्र.सू.रि. सं.): 0003\nDate (दिनांक): 05/01/2019 22:51\n2.\nS.No. (क्र.सं.)\nActs (अधिनियम)\nSections (धारा(एँ))\n1\nARMS ACT, 1959\n25\n2\nARMS ACT, 1959\n54\n3\nARMS ACT, 1959\n59\n3.\n(a)\nOccurrence of offence (अपराध की घटना):\nDate from (दिनांक से): \n1\nDay (दिन): \nDate To (दिनांक तक): \nTime Period (समय अवधि): \nTime To (समय तक): \nTime From (समय से): \n(b)\nInformation received at P.S. (थाना जहां सूचना प्राप्त हुई):\nDate (दिनांक): 05/01/2019\nTime (समय): 18:30 hrs\n(c)\nGeneral Diary Reference (रोजनामचा संदर्भ): \nEntry No. (प्रविष्टि सं.):  020\nTime (समय): 05/01/2019 22:44 hrs\n4.\nType of Information (सूचना का प्रकार): Written\n5.\nPlace of Occurrence (घटनास्थल): \n1.\n(a) Direction and distance 

In [381]:
first_page_data

["HARYANA POLICE CITIZEN SERVICES (हरियाणा पुलिस नागरिक सेवा)\n    FIRST INFORMATION REPORT\n         (Under Section 154 Cr.P.C.)\n        प्रथम सूचना रिपोर्ट\n (धारा 154 दंड प्रक्रिया सहिंता के तहत)\n1.\nDistrict (ज़िला): FARIDABAD\nP.S. (थाना): BHUPANI\nYear (वर्ष): 2019\nFIR No. (प्र.सू.रि. सं.): 0001\nDate (दिनांक): 03/01/2019 20:17\n2.\nS.No. (क्र.सं.)\nActs (अधिनियम)\nSections (धारा(एँ))\n1\nTHE PUBLIC GAMBLING ACT, 1867, 3 OF 1867\n13\n3.\n(a)\nOccurrence of offence (अपराध की घटना):\nDate from (दिनांक से): 03/01/2019\n1\nDay (दिन): Thursday\nDate To (दिनांक तक): 03/01/2019\nTime Period (समय अवधि): Pahar 6\nTime To (समय तक): 16:50 hrs\nTime From (समय से): 16:05 hrs\n(b)\nInformation received at P.S. (थाना जहां सूचना प्राप्त हुई):\nDate (दिनांक): 03/01/2019\nTime (समय): 17:30 hrs\n(c)\nGeneral Diary Reference (रोजनामचा संदर्भ): \nEntry No. (प्रविष्टि सं.):  018\nTime (समय): 03/01/2019 19:54 hrs\n4.\nType of Information (सूचना का प्रकार): Written\n5.\nPlace of Occurrence (घटनास्थल):

In [396]:
corpus = pd.DataFrame(zip(fir_texts, first_page_data), columns=["fir_text", "metadata_html"])
corpus.to_csv("test_files.csv")

In [377]:
driver.close()